# Исследование надежности заемщиков


Во второй части проекта вы выполните шаги 3 и 4. Их вручную проверит ревьюер.
Чтобы вам не пришлось писать код заново для шагов 1 и 2, мы добавили авторские решения в ячейки с кодом. 



## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [ ]:
data.head(20)

**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [ ]:
data.info()

## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [ ]:
data.isna().sum()

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [ ]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [ ]:
data['children'].unique()

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [ ]:
data['children'].unique()

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [ ]:
data.isna().sum()

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [ ]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
data['purpose'].unique()

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Если прямо сильно придерживаться требование в 1 строку, то получим некрасивые проценты

In [ ]:
pd.pivot_table(data,index ='children',values='debt', aggfunc=[len,sum,'mean'])

Для симпотичного вида нужно использовать переменну

In [ ]:
children_pivot = data.pivot_table(index ='children',values='debt', aggfunc=[len,sum,'mean'])
children_pivot['mean'] = round(children_pivot['mean']*100,2)
children_pivot

**Вывод:** По полученной таблице видно, что наличие детей плохо сказывается на показатель возврата кредита в срок.
У клиентов без детей просрочки возникают в 7.5% случаях, а с детьми в  8.18-9.76%.


#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_status_pivot = data.pivot_table(index ='family_status',values='debt', aggfunc=[len,sum,'mean'])
family_status_pivot['mean'] = round(family_status_pivot['mean']*100,2)
family_status_pivot

**Вывод:** Клиенты не обремененные отношениями или состоящие гражданском браке склонны чаще иметь долгки по кредитам, по сравнения с клиентами состоящими или бывшими в узаконненых отношениях.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
income_category_pivot = data.pivot_table(index ='total_income_category',values='debt', aggfunc=[len,sum,'mean'])
income_category_pivot['mean'] = round(income_category_pivot['mean']*100,2)
income_category_pivot

**Вывод:** Т.к. записей в категории А,D,E крайне мало, то полученная статистика получится необъективной. Остается С и B, по данным которых можно увидеть, что чем выше доход, тем процент просрочек ниже.  

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_pivot = data.pivot_table(index ='purpose_category',values='debt', aggfunc=[len,sum,'mean'])
purpose_pivot['mean'] = round(purpose_pivot['mean']*100,2)
purpose_pivot

**Вывод:** Самыми неблагонадежными оказались категории 'операции с автомобилем' и 'получение образования'

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Причин возникновения множество, как технических так и совершенных чловеком. Если рассматривать нашу таблицу, то это колонки days_employed и total_income, которые по какой-то причине совпадают по количеству. Общий трудовой стаж и ежемесечный доход берется из справки 2-НДФЛ. Возможно на одном из серверов произошел сбой, отключение электроэннергии, внеплановый тест оборудования в момент загрузки или обновления данных. Но возможно эти данные собирались из разных источников воедино и человек, ответсвенный за формирование датасета, просто их не прогрузил.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Медианное значение указывает на центральное значение, вокруг которого набор значений обладает тенденцием кластеризации. Получается что-то вроде "экватора" массива значений. 

### Шаг 4: общий вывод.

В 3-ем шаге мы получили данные которые не слишком сильно отличаются друг от друга. Сперва покажется незначительными разница в категориях в 1-2 процента. Но если наложить эти статистические фильтры к потребителю, то можно предложить конкретизированные продукты банка подходящие именно ему (% кредита, максимльная сумма кредита, срок). Чем больше работы проведут аналитики банка, тем выше его конкурентная способность на рынке.                                                                                  
**Дополнение** Судя по полученным результатом можно увидеть картину благонадежного заемщика:                                   <br> *1) Размер дохода B и выше*
<br> *2) Наличие или опыт узаконенных отношений*
<br> *3) Отсутсвие детей*
<br> *4) Предпочтетельны кредитный продукт - операции с недвижимостью*
<br><br>В общем это молодая семья без детей, с хорошим доходом, которая покупает квартиру)